## Введение в МО: каким оно бывает и каковы основные компоненты


# 1
Машинное обучение с учителем подразумевает:

`Обучение на прецедентах - объектах и соответствуюших им ответам.`

# 2

`Вещественными признаками и таргетами (ответами)` называют такие, которые принимают значения исключительно из некоторого установленного множества: например, 0 или 1

`Не верно`

Пояснение: называя признак или ответ вещественным, мы ожидаем, что чем больше данных у нас будет, тем больше различных уникальных значений для наших объектов встретятся в этих признаках или таргетах. Простой пример – признак, описывающий вес человека. При определенной точности замера, практически у всех людей он будет отличаться. А совпадения хоть и будут, но не так часто, как, например, для признака “пол человека”.

# 3

В выборке не может быть пропущенных значений. А если они и встречаются, нам необходимо удалить все объекты, у которых хотя бы один признак (или ответ) отсутствует.

`Не обязательно. Мы можем обрабатывать такие проблемные места. Например, если признак имеет временную структуру, можем заполнять пропуски предыдущими значениями.`

# 4

`Категориальные признаки` часто “декодируют”, чтобы получить некоторый числовой эквивалент текстовому описанию объекта. Например, это можно делать с помощью счетчиков. Где процедура описана наиболее правильно и полно?

`Необходимо понять, как категории между собой отличаются с точки зрения своего влияния на таргет. Посчитаем какую-то аггрегацию каждой категории поверх нашего таргета: например, его среднее значение для каждой группы объектов (категории). Заменим категории на соответствующие им значения. `

# Практика
Описание данных:

`id` - ID поездки

`vendor_id` - ID компании, осуществляющей перевозку

`pickup_datetime` - Таймкод начала поездки

`dropoff_datetime` - Таймкод конца поездки

`passenger_count` - Количество пассажиров

`pickup_longitude` - Долгота точки, в которой началась поездка

`pickup_latitude` - Широта точки, в которой началась поездка

`dropoff_longitude` - Долгота точки, в которой закончилась поездка

`dropoff_latitude` - Широта точки, в которой закончилась поездка

`store_and_fwd_flag- Yes/No:` Была ли информация сохранена в памяти транспортного средства из-за потери соединения с сервером

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

taxiDB = pd.read_csv('taxi_dataset.csv')

taxiDB.head(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N


Наша целевая переменная - `длительность поездки`.

Зная тайм-коды времени начала и конца поездок, можем вычислить обозначенный таргет , производим вычисления в секундах. 
перевод строки в datetime тип, с которым удобно работать при вычленении дней/часов...( `pandas.to_datetime()` )

https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

И этот для перевода разницы datetime объектов в секунды (`dt.total_seconds()` )

https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.total_seconds.html

Положите таргетную переменнул в столбик с названием `trip_duration`

In [2]:
taxiDB['pickup_datetime'] = pd.to_datetime(taxiDB.pickup_datetime)
taxiDB['dropoff_datetime'] = pd.to_datetime(taxiDB.dropoff_datetime)
taxiDB['trip_duration'] = (taxiDB['dropoff_datetime'] - taxiDB['pickup_datetime']).dt.total_seconds()

taxiDB.head(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455.0
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663.0
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124.0


Предсказывая таргет для новых объектов в будущем, мы не будем заранее знать `dropoff_datetime`.

__Удалим__ колонку из датасета.

In [3]:
taxiDB = taxiDB.drop('dropoff_datetime', axis=1)

В будущем будем строить модель. __На каких признаках?__ Рассмотрим имеющиеся вещественные/бинарные и обсудим, какие простейшие признаки можно вытащить из остальных колонок.

- Во-первых, имеем бинарный признак `vendor_id`, принимающий значения `{1, 2}`. Переведем его во множество `{0, 1}`, так как это просто привычнее.

In [4]:
taxiDB['vendor_id'] = taxiDB['vendor_id'] - 1
taxiDB['vendor_id'].value_counts()

1    780302
0    678342
Name: vendor_id, dtype: int64

# 5

`Найдите еще один бинарный признак в данном датасете. Закодируйте и его тоже во множество {0, 1}.`

Давайте проверим, как вы справились с 

- кодировкой бинарных признаков (колонки vendor_id и store_and_fwd_flag)

- Обратите внимание, что у вас при кодировании store_and_fwd_flag 'N' кодируется как 0, а 'Y' как 1

- Выполните необходимые вычисления в ноутбуке. 
- После того, как вы изменили эти колонки, сохраните первые 10 значений полученного промежуточного датафрейма в файл в формате csv с сепаратором ;. Отправьте полученный файл в форму ниже

In [5]:
taxiDB.loc[(taxiDB['store_and_fwd_flag'] == 'N'), 'store_and_fwd_flag'] = 0 # переводим в колонке N->0
taxiDB.loc[(taxiDB['store_and_fwd_flag'] == 'Y'), 'store_and_fwd_flag'] = 1 # переводим в колонке Y->1

In [6]:
# Сохраните первые 10 строк получившегося на этом этапе датафрейма в csv файл с разделителем ';'. Прикрепите в качестве ответа к заданию 6.
taxiDB.head(10).to_csv('ML_1_5.csv', sep=';')
!cat ML_1_5.csv

;id;vendor_id;pickup_datetime;passenger_count;pickup_longitude;pickup_latitude;dropoff_longitude;dropoff_latitude;store_and_fwd_flag;trip_duration
0;id2875421;1;2016-03-14 17:24:55;1;-73.98215484619139;40.76793670654297;-73.96463012695312;40.765602111816406;0;455.0
1;id2377394;0;2016-06-12 00:43:35;1;-73.98041534423827;40.738563537597656;-73.99948120117188;40.73115158081056;0;663.0
2;id3858529;1;2016-01-19 11:35:24;1;-73.97902679443358;40.763938903808594;-74.00533294677734;40.710086822509766;0;2124.0
3;id3504673;1;2016-04-06 19:32:31;1;-74.01004028320312;40.719970703125;-74.01226806640625;40.70671844482422;0;429.0
4;id2181028;1;2016-03-26 13:30:55;1;-73.97305297851561;40.79320907592773;-73.9729232788086;40.782520294189446;0;435.0
5;id0801584;1;2016-01-30 22:01:40;6;-73.98285675048828;40.74219512939453;-73.99208068847656;40.74918365478516;0;443.0
6;id1813257;0;2016-06-17 22:34:59;4;-73.9690170288086;40.75783920288086;-73.95740509033203;40.76589584350586;0;341.0
7;id1324603;1;2016-05-21 

Во-вторых, можем использовать долготу и широту точек старта/завершения поездки, чтобы примерно оценить расстояние между 2 точками.

Сами по себе они, как самостоятельные вещественные признаки, вряд ли способны хорошо объяснять длительность поездки.

Базовая идея состоит в том, чтобы посчитать разность долгот и широт соответственно, то есть:

$$
\delta_{long} = \text{dropoff_longitude} - \text{pickup_longitude}
$$

$$
\delta_{lat} =  \text{dropoff_latitude} - \text{pickup_latitude}
$$

А потом вычислить географическое расстояние между 2 точками по теореме Пифагора:

$$
R = \sqrt{\delta^2_{long} + \delta^2_{lat}}
$$

Мы реализуем данную задумку и вычислим такую вещественную колонку **R**, что, в целом, является хорошим тоном при работе с координатами точек.

Только для начала нужно некоторым образом перевести долготу и широту в километры, обеспечив равенство их мер измерения. Потому что, вообще говоря, *градусная мера* широт и долгот имеет неодинаковую шкалу перевода в километры. Так, если пропустить данную деталь, расстояние **R** будет вычислено неверно, ведь катеты тогда будут иметь разную размерность.

В целом, перевод из долгот и широт в расстояние поездки позволяет нам в будущем проверить зависимость **длительности поездки от километража**, и объяснить ее будет куда проще, чем аналогичную между таргетом и изначальными признаками

https://www.datafix.com.au/BASHing/2018-11-07.html

**Начнем переводить каждую долготу в некоторое относительно километровое выражение**

Соберем список из всех широт (как точек старта, так и конца).

In [7]:
allLat = list(taxiDB['pickup_latitude']) + list(taxiDB['dropoff_latitude'])
allLat[:5]

[40.76793670654297,
 40.738563537597656,
 40.763938903808594,
 40.719970703125,
 40.79320907592773]

__Посчитаем медиану__:

Это некоторое "Центральное значение" в отсортированном массиве всех значений.

Иными словами, такое число, меньше и больше которого примерно равное количество объектов.

In [8]:
medianLat  = sorted(allLat)[int(len(allLat)/2)]
medianLat

40.75431823730469

Итого, для **latitude** колонок получили следующие выражения:

*На сколько примерно километров севернее или южнее (в зависимости от знака) точка находится относительно средней широты*

In [9]:
latMultiplier  = 111.32

taxiDB['pickup_latitude']   = latMultiplier  * (taxiDB['pickup_latitude']   - medianLat)
taxiDB['dropoff_latitude']   = latMultiplier  * (taxiDB['dropoff_latitude']  - medianLat)
taxiDB.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,1,2016-03-14 17:24:55,1,-73.982155,1.516008,-73.964630,1.256121,0,455.0
1,id2377394,0,2016-06-12 00:43:35,1,-73.980415,-1.753813,-73.999481,-2.578912,0,663.0
2,id3858529,1,2016-01-19 11:35:24,1,-73.979027,1.070973,-74.005333,-4.923841,0,2124.0
3,id3504673,1,2016-04-06 19:32:31,1,-74.010040,-3.823568,-74.012268,-5.298809,0,429.0
4,id2181028,1,2016-03-26 13:30:55,1,-73.973053,4.329328,-73.972923,3.139453,0,435.0


In [10]:
allLong = list(taxiDB['pickup_longitude']) + list(taxiDB['dropoff_longitude'])

medianLong  = sorted(allLong)[int(len(allLong)/2)] # -73.98085021972656

longMultiplier = np.cos(medianLat*(np.pi/180.0)) * 111.32 # 84.3266572294738

In [11]:
taxiDB['pickup_longitude'] = longMultiplier * (taxiDB['pickup_longitude'] - medianLong)
taxiDB['dropoff_longitude'] = longMultiplier * (taxiDB['dropoff_longitude'] - medianLong)
taxiDB.head(3)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,1,2016-03-14 17:24:55,1,-0.110015,1.516008,1.367786,1.256121,0,455.0
1,id2377394,0,2016-06-12 00:43:35,1,0.036672,-1.753813,-1.571088,-2.578912,0,663.0
2,id3858529,1,2016-01-19 11:35:24,1,0.153763,1.070973,-2.064547,-4.923841,0,2124.0


- Почему мы вычисляли через медианы: они позволяют нам во время вычисления расстояния преобразовать изначальные longtitude/latitude колонки в "отдаленности точек старта/конца поездок" от медианных точек. 

- Есть подозрение, что медианная для поездок точка города - это, на практике, точка скопления вечерних пробок. Нам может быть вполне важно знать, насколько далеко от такого эпицентра ужаса мы начинаем и заканчиваем поездку (насколько севернее/южнее/...) и выделить поверх этой информации дополнительные признаки.<br>

вычислим географическое расстояние `distance_km:`

In [12]:
# h = √((y2-y1)2 + (x2-x1)2)
distance_km = ( ((taxiDB['dropoff_longitude'] - taxiDB['pickup_longitude']) ** 2)\
                + ((taxiDB['dropoff_latitude'] - taxiDB['pickup_latitude']) ** 2) ) ** 0.5

In [13]:
taxiDB['distance_km'] = ( ((taxiDB['dropoff_longitude'] - taxiDB['pickup_longitude']) ** 2)\
                + ((taxiDB['dropoff_latitude'] - taxiDB['pickup_latitude']) ** 2) ) ** 0.5

In [14]:
taxiDB.head(3)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance_km
0,id2875421,1,2016-03-14 17:24:55,1,-0.110015,1.516008,1.367786,1.256121,0,455.0,1.500479
1,id2377394,0,2016-06-12 00:43:35,1,0.036672,-1.753813,-1.571088,-2.578912,0,663.0,1.807119
2,id3858529,1,2016-01-19 11:35:24,1,0.153763,1.070973,-2.064547,-4.923841,0,2124.0,6.392080


In [15]:
# Уберем старые признаки
taxiDB = taxiDB.drop(['pickup_longitude', 'dropoff_longitude',
                      'pickup_latitude', 'dropoff_latitude'], axis=1)

# 6(step 7)
Давайте проверим, как вы вычислили географическое расстояние `distance_km`

- Выполните необходимые вычисления в ноутбуке. После того, как вы добавили новую колонку `distance_km` 
- и убрали старые признаки `(pickup_longitude,  dropoff_longitude,  pickup_latitude,  dropoff_latitude)`, 
- сохраните первые `10` значений полученного промежуточного датафрейма в файл в формате csv с сепаратором ;. 

- Отправьте полученный файл в форму ниже.



In [16]:
taxiDB.head(10).to_csv('ML_1_6.csv', sep=';')
!cat ML_1_6.csv

;id;vendor_id;pickup_datetime;passenger_count;store_and_fwd_flag;trip_duration;distance_km
0;id2875421;1;2016-03-14 17:24:55;1;0;455.0;1.5004789467430026
1;id2377394;0;2016-06-12 00:43:35;1;0;663.0;1.8071194121139642
2;id3858529;1;2016-01-19 11:35:24;1;0;2124.0;6.392080252604317
3;id3504673;1;2016-04-06 19:32:31;1;0;429.0;1.487154705417205
4;id2181028;1;2016-03-26 13:30:55;1;0;435.0;1.1899254482788029
5;id0801584;1;2016-01-30 22:01:40;6;0;443.0;1.1001071911011968
6;id1813257;0;2016-06-17 22:34:59;4;0;341.0;1.3278523894831544
7;id1324603;1;2016-05-21 07:54:58;1;0;1551.0;5.722426850861646
8;id1301050;0;2016-05-27 23:12:23;1;0;255.0;1.311541055213959
9;id0012891;1;2016-03-10 21:45:01;1;0;1225.0;5.126939275009514


обратим внимание на колонку `passenger_count.`
Она принимает занчения: 

In [17]:
taxiDB['passenger_count'].value_counts()

1    1033540
2     210318
5      78088
3      59896
6      48333
4      28404
0         60
7          3
9          1
8          1
Name: passenger_count, dtype: int64

Какой это признак: вещественный, категориальный, порядковый? 

1. С одной стороны, можно воспринимать его как обычный вещественный признак. Ведь само по себе количество пассажиров (без дополнительной обработки) - это некоторое число, которое может принимать большое количество различных значений.

2. С другой стороны, мы с Вами наверняка знаем, что количество пассажиров от поездки к поездке ограничено. Вряд ли если к нам придут новые данные, мы увидим числа бОльшие, чем у нас в датасете. Тогда рассуждаем следующим образом: раз множество значений признака ограничено, то он категориальный (или, в данном случае, даже порядковый! Ведь у нас могут быть какие-то логичные предположения о том, что количество пассажиров может влиять на модель машины и, соответственно, скорость ее передвижения и скорость поездки!)

3. Какой подход выбрать лучше заранее наверняка не узнаешь. Нужны эксперименты с данными и моделями. Тем не менее, я предлагаю Вам предположить, что данный признак является категориальным, и попробовать отточить навыки кодировки таких фичей!

- Предлагаю реализовать прием с **Mean-target encoding'ом**. 

- Заменим колонку **passenger_count** колонкой **category_encoded**.

In [18]:
# создаем новую колонку category_encoded куда мапим средние сгруппированные значения по passenger_count и числу нашего таргета trip_duration
taxiDB['category_encoded'] = taxiDB['passenger_count'].map(taxiDB.groupby('passenger_count')['trip_duration'].mean())

In [19]:
# удаляем ненужную колонку passenger_count
taxiDB = taxiDB.drop('passenger_count', axis=1)
taxiDB.head(3)

,id,vendor_id,pickup_datetime,store_and_fwd_flag,trip_duration,distance_km,category_encoded
0,id2875421,1,2016-03-14 17:24:55,0,455.0,1.500479,930.399753
1,id2377394,0,2016-06-12 00:43:35,0,663.0,1.807119,930.399753
2,id3858529,1,2016-01-19 11:35:24,0,2124.0,6.392080,930.399753


# 7 (step8)

Вы почти получили финальный датафрейм! Давайте проверим, как выглядит ваш промежуточный датафрейм после того, как вы заменили колонку passenger_count колонкой `category_encoded`.

- Выполните все необходимые вычисления в ноутбуке. 
- Cохраните первые 10 значений полученного промежуточного датафрейма в файл в формате csv с сепаратором ; 
- Отправьте полученный файл в форму ниже.

In [20]:
###  Cохраните первые 10 значений полученного промежуточного датафрейма в файл в формате csv с сепаратором ';' Отправьте полученный файл в форму ответа задания 8.
taxiDB.head(10).to_csv('ML_1_7.csv', sep=';')
!cat ML_1_7.csv

;id;vendor_id;pickup_datetime;store_and_fwd_flag;trip_duration;distance_km;category_encoded
0;id2875421;1;2016-03-14 17:24:55;0;455.0;1.5004789467430026;930.3997532751514
1;id2377394;0;2016-06-12 00:43:35;0;663.0;1.8071194121139642;930.3997532751514
2;id3858529;1;2016-01-19 11:35:24;0;2124.0;6.392080252604317;930.3997532751514
3;id3504673;1;2016-04-06 19:32:31;0;429.0;1.487154705417205;930.3997532751514
4;id2181028;1;2016-03-26 13:30:55;0;435.0;1.1899254482788029;930.3997532751514
5;id0801584;1;2016-01-30 22:01:40;0;443.0;1.1001071911011968;1061.35522313947
6;id1813257;0;2016-06-17 22:34:59;0;341.0;1.3278523894831544;1053.5297493310802
7;id1324603;1;2016-05-21 07:54:58;0;1551.0;5.722426850861646;930.3997532751514
8;id1301050;0;2016-05-27 23:12:23;0;255.0;1.311541055213959;930.3997532751514
9;id0012891;1;2016-03-10 21:45:01;0;1225.0;5.126939275009514;930.3997532751514


Остались две колонки: `id, pickup_datetime`

id можно использовать как обычный идентификатор нашего объекта, поэтому поместите данную колонку в качестве индекса нашей таблички:

In [21]:
taxiDB = taxiDB.set_index('id')

In [22]:
taxiDB.head()

,vendor_id,pickup_datetime,store_and_fwd_flag,trip_duration,distance_km,category_encoded
id,,,,,,
id2875421,1,2016-03-14 17:24:55,0,455.0,1.500479,930.399753
id2377394,0,2016-06-12 00:43:35,0,663.0,1.807119,930.399753
id3858529,1,2016-01-19 11:35:24,0,2124.0,6.392080,930.399753
id3504673,1,2016-04-06 19:32:31,0,429.0,1.487155,930.399753
id2181028,1,2016-03-26 13:30:55,0,435.0,1.189925,930.399753


# 8
выполнены почти все шаги, остался последний: id можно использовать как обычный идентификатор нашего объекта, поэтому поместим данную колонку в качестве индекса нашей таблички

Выполните оставшиеся шаги в ноутбуке. Сохраните первые 10 000 значений полученного датафрейма в файл в формате csv с сепаратором ; Отправьте полученный файл в форму ниже.

In [23]:
taxiDB[:10000].to_csv('ML_1_8.csv', sep=';')

In [24]:
!cat ML_1_8.csv

id;vendor_id;pickup_datetime;store_and_fwd_flag;trip_duration;distance_km;category_encoded
id2875421;1;2016-03-14 17:24:55;0;455.0;1.5004789467430026;930.3997532751514
id2377394;0;2016-06-12 00:43:35;0;663.0;1.8071194121139642;930.3997532751514
id3858529;1;2016-01-19 11:35:24;0;2124.0;6.392080252604317;930.3997532751514
id3504673;1;2016-04-06 19:32:31;0;429.0;1.487154705417205;930.3997532751514
id2181028;1;2016-03-26 13:30:55;0;435.0;1.1899254482788029;930.3997532751514
id0801584;1;2016-01-30 22:01:40;0;443.0;1.1001071911011968;1061.35522313947
id1813257;0;2016-06-17 22:34:59;0;341.0;1.3278523894831544;1053.5297493310802
id1324603;1;2016-05-21 07:54:58;0;1551.0;5.722426850861646;930.3997532751514
id1301050;0;2016-05-27 23:12:23;0;255.0;1.311541055213959;930.3997532751514
id0012891;1;2016-03-10 21:45:01;0;1225.0;5.126939275009514;930.3997532751514
id1436371;1;2016-05-10 22:08:41;0;1274.0;3.8103571083643564;930.3997532751514
id1299289;1;2016-05-15 11:16:11;0;1128.0;3.7775381626030486;105

id0350998;0;2016-03-05 02:10:11;0;815.0;4.921633238721049;930.3997532751514
id3369416;0;2016-02-05 12:05:56;0;983.0;2.4197447571047728;930.3997532751514
id2393936;1;2016-06-02 08:01:42;0;827.0;2.2525247316772745;1070.2321739575864
id1109989;0;2016-06-01 17:01:47;0;909.0;4.189963679433506;930.3997532751514
id2827174;0;2016-03-01 11:31:10;0;1034.0;3.097094919097766;930.3997532751514
id2431582;0;2016-02-25 13:39:33;0;417.0;1.6503103285814755;930.3997532751514
id3682429;1;2016-01-19 06:52:02;0;328.0;1.2730056043315499;930.3997532751514
id3156314;1;2016-06-23 15:08:46;0;3216.0;9.608945149077108;1061.35522313947
id1441848;0;2016-03-27 19:06:30;0;2160.0;7.901765422791072;930.3997532751514
id2402994;0;2016-02-14 21:08:47;0;733.0;2.6633024872473396;930.3997532751514
id2672518;1;2016-02-11 13:23:21;0;1126.0;2.128989345250127;930.3997532751514
id1844241;0;2016-05-27 03:45:27;0;1836.0;11.174096750562667;1028.236276212101
id3651834;1;2016-05-06 19:23:43;0;809.0;1.0477803790956148;1070.2321739575864